# 🏥 MedConsult: Multi-Agent Clinical Co-Pilot with SiriuS Learning
**Google HAI-DEF / MedGemma Impact Challenge**

MedConsult is a multi-agent medical reasoning system driven by Google's **MedGemma** model — fully offline by default. It bridges the gap between raw medical data and patient-friendly explanations by utilizing a specialized team of AI agents:
1. **Analyst (MedGemma)**: Extracts factual medical data.
2. **Clinician (MedGemma)**: Interprets patterns and differential diagnoses.
3. **Critic (MedGemma)**: Translates findings into a plain-language patient summary.
4. **Evaluator & SiriuS (MedGemma)**: Evaluates the reasoning chain, extracts medical lessons, and stores them in ChromaDB for future In-Context Learning (ICL).

> **Cloud toggle**: Pass `eval_backend="gemini"` (requires `GOOGLE_API_KEY`) or `eval_backend="openai"` (requires `OPENAI_API_KEY`) to `MedConsultPipeline()` if you want cloud-based evaluation. Default is `"medgemma"` — no cloud required.

This notebook demonstrates the end-to-end pipeline, visualizes the agent trace, and showcases the SiriuS (Systematic Improvement and Reasoning Injection via Updatable Storage) continuous learning loop.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Setup Environment & Hardware Lock

**Step 1 — Lock Hardware (Already Done)**  
You selected: **GPU T4 x2**

- Do NOT switch to TPU  
- Do NOT switch to CPU  
- Do NOT change accelerator mid-session  

**Step 2 — Lock Software**  
Run the cell below **once** in a fresh Kaggle session. Then **Restart Kernel (MANDATORY)**.


In [4]:
import importlib, sys

def ok(name):
    try:
        m = importlib.import_module(name)
        return True, getattr(m, "__version__", "no_version")
    except Exception as e:
        return False, str(e)

for pkg in ["torch", "transformers", "accelerate", "bitsandbytes", "chromadb", "gradio"]:
    status = ok(pkg)
    print(pkg, "=>", status)

torch => (True, '2.5.1+cu121')
transformers => (True, '5.2.0')
accelerate => (True, '0.30.1')
bitsandbytes => (True, '0.43.3')
chromadb => (True, '1.5.1')
gradio => (False, "cannot import name 'HfFolder' from 'huggingface_hub' (/usr/local/lib/python3.12/dist-packages/huggingface_hub/__init__.py)")


In [5]:
import os, shutil, stat, chromadb

db_path = "/kaggle/temp/chroma_smoke"  # temp is safest writable location on Kaggle
shutil.rmtree(db_path, ignore_errors=True)
os.makedirs(db_path, exist_ok=True)

# force write permissions on folder (and anything created inside)
os.chmod(db_path, 0o777)

client = chromadb.PersistentClient(path=db_path)
col = client.get_or_create_collection("medical_lessons")

col.add(
    ids=["t1"],
    documents=["Rule: For CBC, identify which values are high/low vs reference."],
    metadatas=[{"topic":"cbc","input_type":"lab_report","confidence":"high","source_score":4}]
)

res = col.query(query_texts=["CBC interpretation"], n_results=1)
print("✅ Chroma OK:", res["documents"][0][0])
print("DB path used:", db_path)

✅ Chroma OK: Rule: For CBC, identify which values are high/low vs reference.
DB path used: /kaggle/temp/chroma_smoke


In [2]:
!pip -q install --no-cache-dir -U "transformers>=4.50.0"

In [1]:
import transformers
print("transformers =", transformers.__version__)

transformers = 5.2.0


In [ ]:
%cd /kaggle/working
!rm -rf MedConsult
!git clone https://github.com/ali-amjad52114/MedConsult.git
%cd /kaggle/working/MedConsult/medconsult

# ===== HARD LOCK ENVIRONMENT (GPU T4 x2) =====
!pip -q uninstall -y torch torchvision torchaudio transformers accelerate bitsandbytes

# torch 2.6 is only packaged for cu124 (not cu121).
# Kaggle T4 driver supports CUDA 12.4+, so cu124 wheels run fine.
# transformers>=5.0 requires torch>=2.6 for Gemma attention masks.
!pip -q install --no-cache-dir \
  torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 \
  --index-url https://download.pytorch.org/whl/cu124

# Core stack — required for all modes (MedGemma, chromadb, gradio)
!pip -q install --no-cache-dir \
  "transformers>=5.0.0" \
  "accelerate>=0.34.0" \
  "bitsandbytes>=0.43.3" \
  "sentencepiece==0.2.0" \
  "gradio>=5.0.0" \
  "plotly"

# Install chromadb separately so pip resolves against the locked transformers above
!pip -q install --no-cache-dir chromadb

# Optional — only needed if you use eval_backend="gemini" or eval_backend="openai"
# Uncomment the line(s) below if you want cloud-based evaluation:
# !pip -q install --no-cache-dir "google-generativeai>=0.9.0"
# !pip -q install --no-cache-dir "openai"

# Then: Restart Kernel (MANDATORY)


## 2. Configure HF Token & GPU

After **Restart Kernel**, run the verify cell below, then load the HuggingFace token from Kaggle Secrets.  
**No cloud API key is required** — all agents (including Evaluator & SiriuS) run on MedGemma by default.

GPU T4 with bfloat16 / 4-bit quantization is used for MedGemma.

> **Optional cloud**: If you want Gemini or OpenAI evaluation, add `GOOGLE_API_KEY` or `OPENAI_API_KEY` to Kaggle Secrets and pass `eval_backend="gemini"` or `eval_backend="openai"` when initializing the pipeline.

In [ ]:
import torch, transformers

print("Torch:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")
print("Transformers:", transformers.__version__)

# Expected after lock + restart:
# Torch: 2.6.0+cu121
# CUDA Available: True
# GPU: Tesla T4
# Transformers: 5.x.x

**Step 3 — Never Do These Again**

Once locked:

- Do not run `pip install -U transformers`
- Do not upgrade torch mid-session
- Do not mix CPU + GPU runs
- Do not monkey-patch agents repeatedly
- Do not change accelerator type

If something breaks: restart session, then rerun the lock cell.

In [ ]:
from kaggle_secrets import UserSecretsClient
import os
import torch

# HF_TOKEN is required to download MedGemma from HuggingFace Hub.
try:
    os.environ["HF_TOKEN"] = UserSecretsClient().get_secret("Kaggle_med")
    print("✅ HF_TOKEN loaded.")
except Exception as e:
    print(f"⚠️ Could not load HF_TOKEN: {e}. Ensure 'Kaggle_med' secret is attached.")

# Optional: load cloud API keys only if you want to use eval_backend="gemini" or "openai".
# Leave these out for fully offline MedGemma-only mode.
for secret_name, env_var in [("Gemini API Key", "GOOGLE_API_KEY"), ("OPENAI_API_KEY", "OPENAI_API_KEY")]:
    try:
        os.environ[env_var] = UserSecretsClient().get_secret(secret_name)
        print(f"✅ {env_var} loaded (optional cloud backend available).")
    except Exception:
        pass  # Not present — that's fine for offline mode

print(f"\nGPU available : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name      : {torch.cuda.get_device_name(0)}")
    print(f"GPU memory    : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"bfloat16      : {torch.cuda.is_bf16_supported()}")
else:
    print("WARNING: No GPU — inference will be very slow.")

## 3. Initialize MedConsult Pipeline
We instantiate the `MedConsultPipeline` with the default `eval_backend="medgemma"` — all agents, including the Evaluator and SiriuS components, run entirely on MedGemma with no cloud required.

In [9]:
import os

print("--- Checking Root ---")
os.system("ls -la /kaggle/working/MedConsult")

print("\n--- Checking Inner Folder ---")
os.system("ls -la /kaggle/working/MedConsult/medconsult")

print("\n--- Python Path ---")
import sys
import pprint
pprint.pprint(sys.path)


--- Checking Root ---
total 60
drwxr-xr-x 5 root root  4096 Feb 24 19:05 .
drwxr-xr-x 5 root root  4096 Feb 24 19:05 ..
drwxr-xr-x 2 root root  4096 Feb 24 19:05 .claude
-rw-r--r-- 1 root root  7830 Feb 24 19:05 create_project.py
drwxr-xr-x 8 root root  4096 Feb 24 19:05 .git
-rw-r--r-- 1 root root   415 Feb 24 19:05 .gitignore
drwxr-xr-x 9 root root  4096 Feb 24 19:05 medconsult
-rw-r--r-- 1 root root 21246 Feb 24 19:05 phase4b_out3.txt
-rw-r--r-- 1 root root   317 Feb 24 19:05 run_tests.py

--- Checking Inner Folder ---
total 892
drwxr-xr-x 9 root root   4096 Feb 24 19:05 .
drwxr-xr-x 5 root root   4096 Feb 24 19:05 ..
drwxr-xr-x 2 root root   4096 Feb 24 19:05 agents
-rw-r--r-- 1 root root  10474 Feb 24 19:05 app.py
-rw-r--r-- 1 root root  11022 Feb 24 19:05 ARCHITECTURE.md
-rw-r--r-- 1 root root  10359 Feb 24 19:05 benchmarked_app.py
-rw-r--r-- 1 root root  21019 Feb 24 19:05 benchmarking_suite.py
-rw-r--r-- 1 root root  19158 Feb 24 19:05 cloud_custom_test.txt
-rw-r--r-- 1 root ro

In [ ]:
import importlib.util, sys
print("sys.path[0:6] =", sys.path[0:6])
print("find_spec('medconsult') =", importlib.util.find_spec("medconsult"))
print("find_spec('medconsult.pipeline') =", importlib.util.find_spec("medconsult.pipeline"))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



sys.path[0:6] = ['/kaggle/working', '/kaggle/lib/kagglegym', '/kaggle/lib', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload']
find_spec('medconsult') = None
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_661/2907313623.py", line 4, in <cell line: 0>
    print("find_spec('medconsult.pipeline') =", importlib.util.find_spec("medconsult.pipeline"))
                                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib.util>", line 91, in find_spec
ModuleNotFoundError: No module named 'medconsult'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceb

In [10]:
import os, glob

pkg = "/kaggle/working/MedConsult/medconsult"
print("Listing medconsult/:")
print(sorted(os.listdir(pkg))[:50])

print("\nMatches for pipeline* :")
print(glob.glob(pkg + "/pipeline*"))

print("\nExact exists checks:")
print("pipeline.py exists:", os.path.exists(pkg + "/pipeline.py"))
print("pipeline.py size:", os.path.getsize(pkg + "/pipeline.py") if os.path.exists(pkg + "/pipeline.py") else None)

Listing medconsult/:
['ARCHITECTURE.md', 'README.md', 'TECHNICAL_SPEC.md', 'agents', 'app.py', 'benchmarked_app.py', 'benchmarking_suite.py', 'cloud_custom_test.txt', 'cloud_test_output.txt', 'debug_output.txt', 'download_model.py', 'experience_library', 'model', 'optimized_app.py', 'optimized_pipeline.py', 'out.txt', 'phase4b_out.txt', 'pipeline.py', 'prepopulate_memory.py', 'prompts', 'report.xml', 'report0.xml', 'report1.xml', 'requirements.txt', 'requirements_benchmarking.txt', 'requirements_optimized.txt', 'reset_database.py', 'results', 'run_test.py', 'run_tests_manual.py', 'run_tests_manual_phase1.py', 'sirius', 'test_components.py', 'test_output.txt', 'test_output_4b.txt', 'test_phase0_output.txt', 'tests', 'verify_setup.py', 'visual_dashboard.py']

Matches for pipeline* :
['/kaggle/working/MedConsult/medconsult/pipeline.py']

Exact exists checks:
pipeline.py exists: True
pipeline.py size: 14212
ERROR! Session/line number was not unique in database. History logging moved to new

In [11]:
import os
print(os.listdir("/kaggle/working/MedConsult/medconsult/model"))

['__init__.py', 'cloud_manager.py', 'medgemma_manager.py', 'model_manager.py']


In [ ]:
import importlib.util, sys

MODEL_INIT = "/kaggle/working/MedConsult/medconsult/model/__init__.py"

spec = importlib.util.spec_from_file_location("model", MODEL_INIT)
model_pkg = importlib.util.module_from_spec(spec)
sys.modules["model"] = model_pkg
spec.loader.exec_module(model_pkg)

import model.medgemma_manager as mm
print("✅ Imported:", mm.__file__)
print("MedGemmaManager exists:", hasattr(mm, "MedGemmaManager"))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2044, in __getattr__
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2238, in _get_module
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2236, in _get_module
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1310, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load

In [ ]:
import sys
MEDCONSULT_DIR = "/kaggle/working/MedConsult/medconsult"
if MEDCONSULT_DIR not in sys.path:
    sys.path.insert(0, MEDCONSULT_DIR)

import model.medgemma_manager as mm
print("✅ Imported:", mm.__file__)
print("MedGemmaManager exists:", hasattr(mm, "MedGemmaManager"))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2044, in __getattr__
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2238, in _get_module
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2236, in _get_module
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1310, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load

In [ ]:
import sys, importlib.util

MEDCONSULT_DIR = "/kaggle/working/MedConsult/medconsult"
if MEDCONSULT_DIR not in sys.path:
    sys.path.insert(0, MEDCONSULT_DIR)

path = MEDCONSULT_DIR + "/pipeline.py"
spec = importlib.util.spec_from_file_location("pipeline", path)
pipeline = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pipeline)

print("✅ Loaded pipeline module")
print("Has MedConsultPipeline:", hasattr(pipeline, "MedConsultPipeline"))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2044, in __getattr__
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2238, in _get_module
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/import_utils.py", line 2236, in _get_module
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1310, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load

In [ ]:
import importlib.util, sys

path = "/kaggle/working/MedConsult/medconsult/pipeline.py"

spec = importlib.util.spec_from_file_location("medconsult.pipeline", path)
mod = importlib.util.module_from_spec(spec)
sys.modules["medconsult.pipeline"] = mod
spec.loader.exec_module(mod)

print("Loaded:", mod.__name__)
print("Has MedConsultPipeline:", hasattr(mod, "MedConsultPipeline"))
print("MedConsultPipeline:", getattr(mod, "MedConsultPipeline", None))

In [ ]:
import os, sys, time

PROJECT_ROOT = "/kaggle/working/MedConsult"
PACKAGE_DIR  = os.path.join(PROJECT_ROOT, "medconsult")

print("Exists PROJECT_ROOT?", os.path.exists(PROJECT_ROOT))
print("Exists PACKAGE_DIR?", os.path.exists(PACKAGE_DIR))
print("pipeline exists?", os.path.exists(os.path.join(PACKAGE_DIR, "pipeline.py")))

# 1) Ensure project root is FIRST on sys.path (parent of medconsult/)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# 2) Ensure medconsult is a proper package
init_file = os.path.join(PACKAGE_DIR, "__init__.py")
if not os.path.exists(init_file):
    open(init_file, "w").close()
    print("✅ Created medconsult/__init__.py")

# 3) Clear any cached partial imports
for m in list(sys.modules.keys()):
    if m == "medconsult" or m.startswith("medconsult."):
        del sys.modules[m]

# 4) Now import via package path
from medconsult.pipeline import MedConsultPipeline

print("Loading models (this may take a minute)...")
t0 = time.time()
pipe = MedConsultPipeline()
print(f"✅ Pipeline initialized in {time.time()-t0:.1f} seconds.")

In [ ]:
import os, shutil, sys

# Point ChromaDB to a fresh writable path in /kaggle/working (not inside the repo).
# The repo-relative path "experience_library/chroma_db" becomes read-only between sessions.
CHROMA_PATH = "/kaggle/working/chroma_db"
shutil.rmtree(CHROMA_PATH, ignore_errors=True)
os.makedirs(CHROMA_PATH, exist_ok=True)
os.chmod(CHROMA_PATH, 0o777)
print(f"✅ ChromaDB path ready: {CHROMA_PATH}")

# Patch MemoryStore to use the writable path before pipeline imports it
repo_path = "/kaggle/working/MedConsult/medconsult"
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

from sirius.memory_store import MemoryStore
_orig_init = MemoryStore.__init__
def _patched_init(self, persist_dir=CHROMA_PATH):
    _orig_init(self, persist_dir=persist_dir)
MemoryStore.__init__ = _patched_init
print("✅ MemoryStore patched to use writable path.")

In [ ]:
import os
import sys
import time

# Force Kaggle into the correct downloaded directory
repo_path = "/kaggle/working/MedConsult/medconsult"
if os.path.exists(repo_path):
    os.chdir(repo_path)
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)
else:
    print("⚠️ MedConsult folder not found! Please run Step 1 again to download it.")

from pipeline import MedConsultPipeline

print("Loading models (this may take a minute)...")
t0 = time.time()

# Default: fully offline — MedGemma handles everything including evaluation.
# To use cloud evaluation instead, pass eval_backend="gemini" or eval_backend="openai"
# (requires GOOGLE_API_KEY or OPENAI_API_KEY to be set above).
pipe = MedConsultPipeline()                      # offline MedGemma (default)
# pipe = MedConsultPipeline(eval_backend="gemini")  # cloud Gemini evaluation
# pipe = MedConsultPipeline(eval_backend="openai")  # cloud OpenAI evaluation

print(f"✅ Pipeline initialized in {time.time()-t0:.1f} seconds.")


## 4. Run Medical Analysis Pipeline
Let's provide the system with a sample Complete Blood Count (CBC) report. The Meta-Agent will route this to the Analyst, Clinician, and Critic.


In [ ]:
import re, pathlib
p = pathlib.Path("/kaggle/working/MedConsult/medconsult/model/medgemma_manager.py")
txt = p.read_text()
m = re.search(r'model_id\s*=\s*["\']([^"\']+)["\']', txt)
print("model_id =", m.group(1) if m else "NOT FOUND")

In [ ]:
!pip -q uninstall -y chromadb
!pip -q install --no-cache-dir --force-reinstall chromadb==1.5.1
import chromadb
print("chromadb =", chromadb.__version__)

In [ ]:
import chromadb
print("chromadb =", chromadb.__version__)

In [ ]:
import numpy as np
print("numpy =", np.__version__)

In [ ]:
import shutil, os, glob

# Hugging Face cache locations Kaggle commonly uses
paths = [
    os.path.expanduser("~/.cache/huggingface/hub"),
    "/kaggle/working/.cache/huggingface/hub",
]

model_key = "models--google--medgemma-1.5-4b-it"

deleted = False
for base in paths:
    target = os.path.join(base, model_key)
    if os.path.exists(target):
        shutil.rmtree(target)
        print("✅ Deleted cache:", target)
        deleted = True

if not deleted:
    print("⚠️ Cache folder not found (ok). We'll still force re-download by setting HF_HOME).")

# Optional: isolate fresh cache in working directory (prevents Kaggle collisions)
os.environ["HF_HOME"] = "/kaggle/working/hf_home"
print("HF_HOME set to:", os.environ["HF_HOME"])

In [ ]:
import time
from IPython.display import display, HTML

def md_box(title, text, color="#2196F3"):
    display(HTML(
        f'<div style="border-left: 5px solid {color}; padding: 10px; background-color: #f8f9fa; margin-bottom: 20px;">'
        f'<h4 style="color: {color}; margin-top: 0;">{title}</h4>'
        f'<pre style="white-space: pre-wrap; font-size: 13px;">{text}</pre></div>'
    ))

print("--- PATIENT INPUT ---")
with open("tests/test_data/sample_cbc.txt") as f:
    cbc_text = f.read()
print(cbc_text[:250] + "...\n")

# ---- LIVE AGENT CHAT INTERCEPTOR (SAFE + IDEMPOTENT) ----
# Avoid infinite recursion if you run this cell multiple times.
if not hasattr(pipe, "_orig_methods"):
    pipe._orig_methods = {
        "analyst": pipe.analyst.analyze,
        "clinician": pipe.clinician.interpret,
        "critic": pipe.critic.review_and_communicate,
    }

original_analyst = pipe._orig_methods["analyst"]
original_clinician = pipe._orig_methods["clinician"]
original_critic = pipe._orig_methods["critic"]

def analyst_live(*args, **kwargs):
    print("⏳ 🔬 Analyst Agent is reading the raw data...")
    start = time.time()
    res = original_analyst(*args, **kwargs)
    print(f"✅ Analyst finished in {time.time()-start:.1f} seconds!")
    md_box("🔬 Analyst Output (Sent to Clinician)", res, "#4CAF50")
    return res

def clinician_live(user_text, analyst_output, *args, **kwargs):
    print("⏳ 🏥 Clinician Agent is interpreting the Analyst's work...")
    start = time.time()
    res = original_clinician(user_text, analyst_output, *args, **kwargs)
    print(f"✅ Clinician finished in {time.time()-start:.1f} seconds!")
    md_box("🏥 Clinician Output (Sent to Critic)", res, "#FF9800")
    return res

def critic_live(user_text, analyst_output, clinician_output, *args, **kwargs):
    print("⏳ 📋 Critic Agent is translating everything for the patient...")
    start = time.time()
    res = original_critic(user_text, analyst_output, clinician_output, *args, **kwargs)
    print(f"✅ Critic finished in {time.time()-start:.1f} seconds!")
    md_box("📋 Final Critic Summary", res, "#9C27B0")
    return res

# Apply patch (safe because originals are stored once)
pipe.analyst.analyze = analyst_live
pipe.clinician.interpret = clinician_live
pipe.critic.review_and_communicate = critic_live
# ---------------------------------------

print("🚀 Starting MedConsult Pipeline...")
result_cbc = pipe.run(cbc_text)
print("\n🎉 Pipeline Complete!")


## 5. Visualizing the Agent Trace
This customized view shows exactly what context was retrieved from memory, how each agent processed the data, and the final patient-friendly communication.


In [ ]:
cbc_text = "CBC: WBC 12.8, RBC 4.2, Hgb 11.2, Hct 33.8. Ref: WBC 4-11, RBC 4.5-5.5, Hgb 13.5-17.5, Hct 38.5-50."

print("🚀 Running pipeline...")
result_cbc = pipe.run(cbc_text)
print("✅ Done. Keys:", list(result_cbc.keys()))

In [ ]:
from IPython.display import display, Markdown, HTML
from sirius.memory_retriever import MemoryRetriever

def md(text): display(Markdown(text))
def section(title, icon="📌"): display(HTML(f'<h2 style="background:#1a1a2e;color:#16c79a;padding:12px;border-radius:8px;margin-top:20px">{icon} {title}</h2>'))
def code_block(text, max_chars=800):
    text = str(text) if text else "None"
    truncated = text[:max_chars] + ("..." if len(text) > max_chars else "")
    display(HTML(f'<pre style="background:#0d1117;color:#c9d1d9;padding:12px;border-radius:8px;overflow-x:auto;font-size:12px;line-height:1.5;white-space:pre-wrap">{truncated}</pre>'))
def metric_box(label, value, color="#16c79a"):
    display(HTML(f'<div style="display:inline-block;background:#1a1a2e;border-left:4px solid {color};padding:8px 16px;margin:4px 8px 4px 0;border-radius:4px"><span style="color:#888;font-size:12px">{label}</span><br><span style="color:{color};font-size:20px;font-weight:bold">{value}</span></div>'))

display(HTML('<h1 style="text-align:center;background:linear-gradient(135deg,#1a1a2e,#16213e);color:#16c79a;padding:20px;border-radius:12px;margin-bottom:20px">🔬 MedConsult — Full Agent Trace</h1>'))

# Memory
section("STEP 0 — Memory Retrieval", "🧠")
retriever = MemoryRetriever(pipe.memory_store)
input_type = pipe.experience_library.classify_input_type(cbc_text)
analyst_memory = analyst_memory = retriever.get_relevant_lessons(cbc_text, agent_name="analyst")
metric_box("Lessons Found", "Yes" if analyst_memory else "None")
if analyst_memory: code_block(analyst_memory, 500)

# Analyst
section("STEP 1 — Analyst Agent (MedGemma 4B)", "🔬")
md("**Output:** Organized Extraction")
code_block(result_cbc.get("analyst", ""), 600)

# Clinician
section("STEP 2 — Clinician Agent (MedGemma 4B)", "🏥")
md("**Output:** Clinical Interpretation")
code_block(result_cbc.get("clinician", ""), 800)

# Critic
section("STEP 3 — Critic Agent (MedGemma 4B)", "📋")
md("**Output:** Patient-Friendly Summary")
code_block(result_cbc.get("critic", ""), 800)


## 6. SiriuS Learning Loop (Continuous Improvement)
Once the clinical agents complete their work, the **Evaluator Agent** reviews the entire chain for quality. If the chain is high-quality, the **Lesson Extractor** distills reasoning patterns and stores them into the vector database. If it's low quality, the **Augmentation Loop** retries the generation with injected feedback.


In [ ]:
print("Evaluating quality and extracting lessons...")
sirius_result = pipe.evaluate_and_learn(result_cbc)

score = sirius_result.get('evaluation', {}).get('score', 0)
print(f"Evaluator Score: {score}/5")

lessons = sirius_result.get('lessons_extracted', 0)
print(f"New Lessons Extracted: {lessons}")

print("\nEvaluator Feedback:")
print(sirius_result.get("evaluation", {}).get("feedback", "No feedback.")[:500])


## 7. Performance & Capabilities Dashboards
Let's visualize the pipeline's execution time, the latest quality score, and the growth of the lesson library.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 4, figsize=(22, 5))

# Plot 1: Timing
timings = result_cbc.get("timings", {})
agents = ["Memory", "Analyst", "Clinician", "Critic"]
times = [timings.get("memory_retrieval", 0), timings.get("analyst", 0), timings.get("clinician", 0), timings.get("critic", 0)]
axes[0].barh(agents, times, color=["#4CAF50", "#2196F3", "#FF9800", "#9C27B0"])
axes[0].set_title("Agent Execution Time")
axes[0].set_xlabel("Seconds")
for i, v in enumerate(times): axes[0].text(v, i, f" {v:.1f}s", va='center')

# Plot 2: Quality Score Gauge
score_val = float(score) if str(score).isdigit() else 0
ax2 = axes[1]
theta = np.linspace(0, np.pi, 100)
ax2.plot(np.cos(theta), np.sin(theta), color="#E0E0E0", linewidth=25)
fill = np.linspace(np.pi, np.pi - (score_val/5)*np.pi, 100)
ax2.plot(np.cos(fill), np.sin(fill), color="#16c79a", linewidth=25)
ax2.text(0, 0, f"{score_val}/5", ha="center", va="center", fontsize=30, fontweight="bold", color="#16c79a")
ax2.set_title("Quality Score")
ax2.set_xlim(-1.2, 1.2); ax2.set_ylim(-0.2, 1.2)
ax2.axis("off")

# Plot 3: Memory Growth
total_lessons = pipe.memory_store.get_lesson_count()
axes[2].bar(["Lessons Extracted\n(This Run)", "Total Lessons\n(Cumulative)"], [lessons, total_lessons], color=["#9b59b6", "#e74c3c"])
axes[2].set_title("SiriuS Memory Growth")
for i, v in enumerate([lessons, total_lessons]): axes[2].text(i, v + 0.1, str(v), ha="center", fontweight="bold")

# Plot 4: Augmentation Frequency
aug = int(sirius_result.get("augmented", False))
axes[3].pie([1 if aug else 0, 1 if not aug else 0], labels=["Augmented", "One-Pass Validation"],
            colors=["#FF9800", "#4CAF50"], autopct="%1.0f%%", startangle=90)
axes[3].set_title("Augmentation Check")

plt.tight_layout()
plt.show()


## 8. Multi-Modal Execution (Image + Text)
MedGemma natively supports vision. Let's process a Chest X-ray alongside clinical text.


In [ ]:
from PIL import Image, ImageDraw
import urllib.request
from pathlib import Path
import time
from IPython.display import display, Markdown, HTML

def md_box(title, text, color="#2196F3"):
    display(HTML(f'<div style="border-left: 5px solid {color}; padding: 10px; background-color: #f8f9fa; margin-bottom: 20px;">'
                 f'<h4 style="color: {color}; margin-top: 0;">{title}</h4><pre style="white-space: pre-wrap; font-size: 13px;">{text}</pre></div>'))

# 1. Create or download the X-Ray
xray_path = Path("/kaggle/working/sample_xray.jpg")
if not xray_path.exists():
    url = "https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/images/auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Biden_New-corona-1.jpeg"
    try:
        urllib.request.urlretrieve(url, xray_path)
    except:
        img = Image.new("RGB", (512, 512), color=(30, 30, 30))
        draw = ImageDraw.Draw(img)
        draw.ellipse([60, 80, 220, 400], fill=(55, 55, 55))
        draw.ellipse([290, 80, 450, 400], fill=(55, 55, 55))
        img.save(xray_path)

xray_img = Image.open(xray_path).convert("RGB")
display(xray_img.resize((256, 256)))

input_text = "Patient presents with shortness of breath. Chest X-ray provided."
print("🚀 Starting Multi-Modal Image Analysis (Under the Hood)...\n")

# --- Step-by-Step Execution ---

# 1. Analyst
print("⏳ [0:00] 🔬 Analyst Agent is scanning the X-Ray and clinical text (1-2 mins)...")
start = time.time()
analyst_output = pipe.analyst.analyze(input_text, image=xray_img)
print(f"✅ Scanning finished in {time.time()-start:.1f} seconds!")
md_box("🔬 Analyst Image Findings", analyst_output, "#4CAF50")

# 2. Clinician
print("⏳ 🏥 Clinician Agent is interpreting the visual findings (1-2 mins)...")
start = time.time()
clinician_output = pipe.clinician.interpret(input_text, analyst_output, image=xray_img)
print(f"✅ Interpretation finished in {time.time()-start:.1f} seconds!")
md_box("🏥 Clinician Diagnosis", clinician_output, "#FF9800")

# 3. Critic
print("⏳ 📋 Critic Agent is translating the diagnosis for the patient (1-2 mins)...")
start = time.time()
critic_output = pipe.critic.review_and_communicate(input_text, analyst_output, clinician_output, image=xray_img)
print(f"✅ Translation finished in {time.time()-start:.1f} seconds!")
md_box("📋 Final Patient Summary", critic_output, "#9C27B0")

print("\n🎉 Multi-Modal Pipeline Complete!")


## 9. Launch the Interactive Gradio Interface
Finally, we can launch the end-to-end user interface directly from this notebook. It provides tabs for Chat, History, Developer Logs, and Validation.


In [ ]:
import threading
import time
from app import launch_ui

def start_gradio():
    launch_ui()

# Launch on a background thread so the notebook remains interactive
t = threading.Thread(target=start_gradio, daemon=True)
t.start()

print("Gradio app is starting... Click the public URL above when it appears.")
